<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/BasesDeDatos/2_SQL/sql_practica_solucion.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a>

# SQL Práctica I

In [1]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql

import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine('postgresql+psycopg2://@/postgres')

def exec_sql(query):
  text_query = sqlalchemy.sql.text(query)

  with engine.connect() as conn:
    res = conn.execute(text_query)
    if query.strip().lower().startswith('select'):
      return pd.DataFrame(res)
    else:
      conn.commit()

 * Starting PostgreSQL 12 database server
   ...done.
CREATE ROLE


### Crear base de datos

In [2]:
exec_sql('CREATE SCHEMA IF NOT EXISTS humai')

## DDL - Crear tablas

### Tabla Customers



```
CREATE TABLE IF NOT EXISTS humai.Customers (
  customer_id INT NOT NULL,
  customer_name VARCHAR(50) NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE,
  PRIMARY KEY (customer_id)
  );
```



In [3]:
create_customers = """
CREATE TABLE IF NOT EXISTS humai.Customers (
  customer_id INT NOT NULL,
  customer_name VARCHAR(50) NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE,
  PRIMARY KEY (customer_id));
"""

exec_sql(create_customers)

### Tabla Orders



```
CREATE TABLE IF NOT EXISTS humai.Orders (
  order_id INT NOT NULL,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  order_price DECIMAL(8,2),
  PRIMARY KEY (order_id),
FOREIGN KEY (customer_id) REFERENCES humai.Customers(customer_id)
  );
```



In [4]:
create_orders = """
CREATE TABLE IF NOT EXISTS humai.Orders (
  order_id INT NOT NULL,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  order_price DECIMAL(8,2),
  PRIMARY KEY (order_id),
  FOREIGN KEY (customer_id) REFERENCES humai.Customers(customer_id)
);
"""

exec_sql(create_orders)

### Tabla Shipments

```
CREATE TABLE IF NOT EXISTS humai.Shipments (
  shipment_id INT NOT NULL,
  order_id INT NOT NULL,
  shipment_date DATE NOT NULL,
  shipment_city VARCHAR(50),
  PRIMARY KEY (shipment_id),
  FOREIGN KEY (order_id) REFERENCES humai.Orders(order_id)
  );

```



In [5]:
create_shipments = """
CREATE TABLE IF NOT EXISTS humai.Shipments (
  shipment_id INT NOT NULL,
  order_id INT NOT NULL,
  shipment_date DATE NOT NULL,
  shipment_city VARCHAR(50),
  PRIMARY KEY (shipment_id),
  FOREIGN KEY (order_id) REFERENCES humai.Orders(order_id)
);
"""

exec_sql(create_shipments)

### Check que se hayan creado las tablas

In [6]:
exec_sql("SELECT * FROM information_schema.tables WHERE table_schema = 'humai';")

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,postgres,humai,customers,BASE TABLE,None,None,None,None,None,YES,NO,None
1,postgres,humai,orders,BASE TABLE,None,None,None,None,None,YES,NO,None
2,postgres,humai,shipments,BASE TABLE,None,None,None,None,None,YES,NO,None


## DML - Insertar datos ficticios



```
INSERT INTO humai.Customers
VALUES
(1, 'Eugenio', '08/21/1998', Null),
(2, 'Mario', '05/05/2005', Null),
(3, 'Pedro', '03/08/2020', '02/05/2022')
;
```



In [7]:
insert_customers = """
INSERT INTO humai.Customers VALUES
  (1, 'Eugenio', '08/21/1998', Null),
  (2, 'Mario', '05/05/2005', Null),
  (3, 'Pedro', '03/08/2020', '02/05/2022')
"""

exec_sql(insert_customers)



```
INSERT INTO humai.Orders
VALUES
(1, 1, '06/05/2022', 45),
(2, 1, '06/05/2021', 60),
(3, 1, '06/06/2022', 70),
(4, 2, '01/05/2022', 5),
(5, 3, '06/10/2022', 145),
(6, 3, '03/02/2022', 2)
;
```



In [8]:
insert_orders = """
INSERT INTO humai.Orders VALUES
  (1, 1, '06/05/2022', 45),
  (2, 1, '06/05/2021', 60),
  (3, 1, '06/06/2022', 70),
  (4, 2, '01/05/2022', 5),
  (5, 3, '06/10/2022', 145),
  (6, 3, '03/02/2022', 2);
"""

exec_sql(insert_orders)


```
INSERT INTO humai.Shipments
VALUES
(1, 1, '06/06/2022', 'Belgrano'),
(2, 2, '06/06/2021', 'Mar del Plata'),
(3, 3, '06/10/2022', 'Belgrano'),
(4, 4, '02/05/2022', 'San Isidro'),
(5, 5, '06/15/2022', 'Belgrano'),
(6, 6, '03/05/2022', 'Mar del Plata')
;
```





In [9]:
insert_shipments = """
INSERT INTO humai.Shipments VALUES
  (1, 1, '06/06/2022', 'Belgrano'),
  (2, 2, '06/06/2021', 'Mar del Plata'),
  (3, 3, '06/10/2022', 'Belgrano'),
  (4, 4, '02/05/2022', 'San Isidro'),
  (5, 5, '06/15/2022', 'Belgrano'),
  (6, 6, '03/05/2022', 'Mar del Plata');
"""

exec_sql(insert_shipments)

## DDL - Consulas SELECT

1. Retorna todos los registros de la tabla Customers.



```
SELECT *
FROM humai.Customers;
```



In [10]:
exec_sql('SELECT * FROM humai.Customers;')

,customer_id,customer_name,fecha_inicio,fecha_fin
0,1,Eugenio,1998-08-21,None
1,2,Mario,2005-05-05,None
2,3,Pedro,2020-03-08,2022-02-05


2. Retorna la cantidad de envios por shipment_city



```
SELECT shipment_city as "Barrio", count(*) as "Envios"
FROM humai.Shipments
GROUP BY shipment_city
```



In [11]:
select_shipment = 'SELECT shipment_city as "Barrio", count(*) as "Envios" FROM humai.Shipments  GROUP BY shipment_city'

exec_sql(select_shipment)

,Barrio,Envios
0,Belgrano,3
1,Mar del Plata,2
2,San Isidro,1


3. Retorna la cantidad de envios a Belgrano



```
SELECT count(*) as "Envios a Belgrano"
FROM humai.Shipments  
WHERE shipment_city = 'Belgrano'
```



In [12]:
exec_sql("SELECT count(*) as \"Envios a Belgrano\" FROM humai.Shipments  WHERE shipment_city = 'Belgrano';")

,Envios a Belgrano
0,3


4. Retorna todas las ordenes mayores a $50



```
SELECT *
FROM humai.Orders
WHERE order_price > 50;
```



In [13]:
exec_sql("SELECT * FROM humai.Orders WHERE order_price > 50;")

,order_id,customer_id,order_date,order_price
0,2,1,2021-06-05,60.00
1,3,1,2022-06-06,70.00
2,5,3,2022-06-10,145.00


5. Retorna el cliente que más dinero gasto



```
SELECT customer_id, sum(order_price) as "Total gastado"
FROM humai.Orders
GROUP BY customer_id
ORDER BY sum(order_price) DESC
LIMIT 1
;
```



In [14]:
exec_sql("SELECT customer_id, sum(order_price) as \"Total gastado\" FROM humai.Orders GROUP BY customer_id ORDER BY sum(order_price) DESC LIMIT 1;")

,customer_id,Total gastado
0,1,175.00


6. Y que si queremos ver el nombre del cliente? Dato que se encuentra en otra tabla.



```
SELECT c.customer_name, sum(order_price) as "Total gastado"
FROM humai.Orders o
FULL JOIN humai.Customers c
ON o.customer_id = c.customer_id
GROUP BY c.customer_name
ORDER BY sum(order_price) DESC
LIMIT 1;
```



In [15]:
select = """
SELECT c.customer_name, sum(order_price) as \"Total gastado\"
  FROM humai.Orders o
  FULL JOIN humai.Customers c
  ON o.customer_id = c.customer_id
  GROUP BY c.customer_name
  ORDER BY sum(order_price) DESC
  LIMIT 1;
"""

exec_sql(select)

,customer_name,Total gastado
0,Eugenio,175.00


7. Retornar la duración promedio de los clientes que se dieron de baja.



```
SELECT CAST(AVG(fecha_fin-fecha_inicio) AS INT) AS "Promedio Total dias"
FROM humai.Customers
WHERE fecha_fin IS NOT NULL
```



In [16]:
exec_sql("SELECT CAST(AVG(fecha_fin-fecha_inicio) AS INT) AS \"Promedio Total dias\" FROM humai.Customers WHERE fecha_fin IS NOT NULL")

,Promedio Total dias
0,699


8. Retornar los clientes que tienen una 'e' en el nombre.


```
SELECT customer_name
FROM humai.Customers
WHERE customer_name LIKE '%e%';

```



In [17]:
exec_sql("SELECT customer_name FROM humai.Customers WHERE customer_name LIKE '%e%';")

,customer_name
0,Eugenio
1,Pedro


## Ejercicios online:

**DML:**
1. Ingresar dos nuevos clientes en la tabla Customers.
2. Actualizar el valor de la orden con order_id 5 a $200.
3. Eliminar uno de los clientes agregados a la tabla Customers.
4. Insertar una orden y su envio.

**DDL:**


1. Calcular la cantidad de ventas realizadas en el 2022.
2. Retornar la fecha de la primera venta registrada.
3. Retornar la venta de mayor dinero de cada cliente que tenga ventas.
4. Retornar la cantidad de envios a cada ciudad que no sea Belgrano.
5. Calcular la cantidad de dias activos que tiene cada cliente que aún no se han dado de baja.
6. Calcular el promedio de cantidad de dias activos que tienen los clientes que aún no se han dado de baja.
7. Retornar la cantidad de envios realizados entre enero y junio del 2022 para Mar del Plata.
8. Retornar la cantidad gastada por cliente con sus nombres para aquellos clientes que hayan gastado mas de $20.





### **DML:**
1. Ingresar dos nuevos clientes en la tabla Customers.

```
INSERT INTO humai.Customers
VALUES
(4, 'Juan', '10/10/1990', Null),
(5, 'Maria', '05/20/2005', '05/20/2015')
;
```

2. Actualizar el valor de la orden con order_id 5 a $200.

```
UPDATE humai.Orders
SET order_price = 200
WHERE order_id = 5
;
```

3. Eliminar uno de los clientes agregados a la tabla Customers.

```
DELETE FROM humai.Customers
WHERE customer_id = 4
;
```

4. Insertar una orden y su envio.


```
INSERT INTO humai.Orders
VALUES
(7, 5, '06/24/2022', 1000)
;

INSERT INTO humai.Shipments
VALUES
(7, 7, '06/26/2022', 'Belgrano')
;
```

In [18]:
# Ingresar dos nuevos clientes en la tabla Customers.
exec_sql("INSERT INTO humai.Customers VALUES (4, 'Juan', '10/10/1990', Null),(5, 'Maria', '05/20/2005', '05/20/2015');")

Reviso que se hayan insertado correctamente:

In [21]:
exec_sql("SELECT * FROM humai.Customers")

,customer_id,customer_name,fecha_inicio,fecha_fin
0,1,Eugenio,1998-08-21,None
1,2,Mario,2005-05-05,None
2,3,Pedro,2020-03-08,2022-02-05
3,4,Juan,1990-10-10,None
4,5,Maria,2005-05-20,2015-05-20


In [22]:
# Actualizar el valor de la orden con order_id 5 a $200.
exec_sql("UPDATE humai.Orders SET order_price = 200 WHERE order_id = 5;")

Reviso que se haya actualizado correctamente:

In [23]:
exec_sql("SELECT * FROM humai.Orders WHERE order_id = 5")

,order_id,customer_id,order_date,order_price
0,5,3,2022-06-10,200.00


In [24]:
#Eliminar uno de los clientes agregados a la tabla Customers.
exec_sql("DELETE FROM humai.Customers WHERE customer_id = 4;")

Reviso que se haya borrado correctamente:

In [25]:
exec_sql("SELECT * FROM humai.Customers")

,customer_id,customer_name,fecha_inicio,fecha_fin
0,1,Eugenio,1998-08-21,None
1,2,Mario,2005-05-05,None
2,3,Pedro,2020-03-08,2022-02-05
3,5,Maria,2005-05-20,2015-05-20


In [26]:
#Insertar una orden y su envio.
exec_sql("INSERT INTO humai.Orders VALUES (7, 5, '06/24/2022', 1000);")

In [27]:
exec_sql("INSERT INTO humai.Shipments VALUES (7, 7, '06/26/2022', 'Belgrano');")

Reviso que se hayan creado correctamente:

In [28]:
exec_sql("SELECT * FROM humai.Orders")

,order_id,customer_id,order_date,order_price
0,1,1,2022-06-05,45.00
1,2,1,2021-06-05,60.00
2,3,1,2022-06-06,70.00
3,4,2,2022-01-05,5.00
4,6,3,2022-03-02,2.00
5,5,3,2022-06-10,200.00
6,7,5,2022-06-24,1000.00


In [29]:
exec_sql("SELECT * FROM humai.Shipments")

,shipment_id,order_id,shipment_date,shipment_city
0,1,1,2022-06-06,Belgrano
1,2,2,2021-06-06,Mar del Plata
2,3,3,2022-06-10,Belgrano
3,4,4,2022-02-05,San Isidro
4,5,5,2022-06-15,Belgrano
5,6,6,2022-03-05,Mar del Plata
6,7,7,2022-06-26,Belgrano


### **DDL:**
1. Calcular la cantidad de ventas realizadas en el 2022.


```
SELECT count(*)
FROM humai.Orders
WHERE EXTRACT(YEAR FROM order_date) = 2022;
```

In [31]:
exec_sql("SELECT count(*) AS \"Ventas en 2022\" FROM humai.Orders WHERE EXTRACT(YEAR FROM order_date) = 2022;")

,Ventas en 2022
0,6


2. Retornar la fecha de la primera venta registrada.


```
SELECT order_date AS "Fecha primera venta"
FROM humai.Orders
ORDER BY order_date ASC
LIMIT 1
;
```



In [32]:
exec_sql("SELECT order_date AS \"Fecha primera venta\" FROM humai.Orders ORDER BY order_date ASC LIMIT 1;")

,Fecha primera venta
0,2021-06-05


3. Retornar la venta de mayor dinero de cada cliente que tenga ventas.


```
SELECT customer_id, MAX(order_price) as "Mayor Venta"
FROM humai.Orders
GROUP BY customer_id
;
```



In [33]:
exec_sql("SELECT customer_id, MAX(order_price) as \"Mayor Venta\" FROM humai.Orders GROUP BY customer_id;")

,customer_id,Mayor Venta
0,3,200.00
1,5,1000.00
2,2,5.00
3,1,70.00


4. Retornar la cantidad de envios a cada ciudad que no sea Belgrano.


```
SELECT shipment_city, count(shipment_id) as n_shipments
FROM Humai.Shipments
WHERE shipment_city != 'Belgrano'
GROUP BY shipment_city
;
```

In [34]:
query = """
SELECT shipment_city, count(shipment_id) as n_shipments
FROM Humai.Shipments
WHERE shipment_city != 'Belgrano'
GROUP BY shipment_city
;
"""

exec_sql(query)

,shipment_city,n_shipments
0,Mar del Plata,2
1,San Isidro,1


5. Calcular la cantidad de dias activos que tiene cada cliente que aún no se han dado de baja.


```
SELECT
customer_name,
CAST(date_part('day', NOW() - fecha_inicio) AS INTEGER) AS active_days
FROM Humai.Customers
;
```

In [35]:
query = """
SELECT
customer_name,
CAST(date_part('day', NOW() - fecha_inicio) AS INTEGER) AS active_days
FROM Humai.Customers
;
"""

exec_sql(query)

,customer_name,active_days
0,Eugenio,9091
1,Mario,6642
2,Pedro,1221
3,Maria,6627


6. Calcular el promedio de cantidad de dias activos que tienen los clientes que aún no se han dado de baja.


```
SELECT
AVG(CAST(date_part('day', NOW() - fecha_inicio) AS INTEGER))
AS "average_active_days"
FROM Humai.Customers
WHERE fecha_fin IS Null
;
```

In [36]:
query = """
SELECT
AVG(CAST(date_part('day', NOW() - fecha_inicio) AS INTEGER))
AS "average_active_days"
FROM Humai.Customers
WHERE fecha_fin IS Null
;
"""

exec_sql(query)

,average_active_days
0,7866.5000000000000000


7. Retornar la cantidad de envios realizados entre enero y junio del 2022 para Mar del Plata.


```
SELECT COUNT(shipment_id) AS n_shipments_mdq
FROM Humai.Shipments
WHERE shipment_date >= '2022-01-01'
AND shipment_date <= '2022-05-31'
AND shipment_city = 'Mar del Plata'
;
```

In [37]:
query = """
SELECT COUNT(shipment_id) AS n_shipments_mdq
FROM Humai.Shipments
WHERE shipment_date >= '2022-01-01'
AND shipment_date <= '2022-05-31'
AND shipment_city = 'Mar del Plata'
;
"""

exec_sql(query)

,n_shipments_mdq
0,1


8. Retornar la cantidad gastada por cliente con sus nombres para aquellos clientes que hayan gastado mas de $20.


```
SELECT customer_name, SUM(order_price) AS total_order_price
FROM Humai.Customers
INNER JOIN Humai.Orders
ON Humai.Customers.customer_id = Humai.Orders.customer_id
WHERE order_price > 20
GROUP BY Humai.Customers.customer_id
;
```

In [38]:
query = """
SELECT customer_name, SUM(order_price) AS total_order_price
FROM Humai.Customers
INNER JOIN Humai.Orders
ON Humai.Customers.customer_id = Humai.Orders.customer_id
WHERE order_price > 20
GROUP BY Humai.Customers.customer_id
;
"""

exec_sql(query)

,customer_name,total_order_price
0,Pedro,200.00
1,Maria,1000.00
2,Eugenio,175.00
